# Preparación de datos de test

In [ ]:
import pickle as pckl

In [ ]:
input_path = "/home/ubuntu/tfm/TrainYourOwnYOLO/Data/Source_Images/Training_Images/vott-csv-export-augmented-check/data_train.txt"
output_path = "/home/ubuntu/tfm/TrainYourOwnYOLO/Data/Source_Images/Training_Images/vott-csv-export-augmented-check/data_test.pckl"
mapper = {1:'Panel', 0:'Dedo'}

In [ ]:
rows = []
with open(input_path) as fd:
    for item in fd:
        filename_and_boxes = item.rstrip('\n').split(' ')
        filename = filename_and_boxes[0]
        boxes = filename_and_boxes[1:]
        d = {'filename': filename, 'object':[]}
        for box in boxes:
            box = box.split(',')
            d['object'].append({'xmin':int(box[0]), 'ymin':int(box[1]), 'xmax': int(box[2]), 'ymax': int(box[3]), 'name': mapper[int(box[4])]})
        rows.append(d)

In [ ]:
pckl.dump(rows, open(output_path, 'wb'))

# Dependencias

In [1]:
import argparse
import json
import pickle as pckl
import numpy as np
import os
import cv2
import pandas as pd
from PIL import Image
from scipy.special import expit
from yolo3.yolo import YOLO

## Cargar modelo

In [2]:
def load_model(model_path, classes_path, anchors_path):

    yolo = YOLO(
        **{
            "model_path": model_path,
            "anchors_path": anchors_path,
            "classes_path": classes_path,
            "score": 0.5,
            "gpu_num": 1,
            "model_image_size": (416, 416),
        }
    )

    return yolo

## Bounding boxes

In [3]:
class BoundBox:
    def __init__(self, xmin, ymin, xmax, ymax, c = None, classes = None):
        self.xmin = xmin
        self.ymin = ymin
        self.xmax = xmax
        self.ymax = ymax
        
        self.c       = c
        self.classes = classes

        self.label = -1
        self.score = -1

    def get_label(self):
        if self.label == -1:
            self.label = np.argmax(self.classes)
        
        return self.label
    
    def get_score(self):
        if self.score == -1:
            self.score = self.classes[self.get_label()]
            
        return self.score 

def _interval_overlap(interval_a, interval_b):
    x1, x2 = interval_a
    x3, x4 = interval_b

    if x3 < x1:
        if x4 < x1:
            return 0
        else:
            return min(x2,x4) - x1
    else:
        if x2 < x3:
             return 0
        else:
            return min(x2,x4) - x3  
        
def bbox_iou(box1, box2):
    intersect_w = _interval_overlap([box1.xmin, box1.xmax], [box2.xmin, box2.xmax])
    intersect_h = _interval_overlap([box1.ymin, box1.ymax], [box2.ymin, box2.ymax])  
    
    intersect = intersect_w * intersect_h

    w1, h1 = box1.xmax-box1.xmin, box1.ymax-box1.ymin
    w2, h2 = box2.xmax-box2.xmin, box2.ymax-box2.ymin
    
    union = w1*h1 + w2*h2 - intersect
    
    return float(intersect) / union

## Generador de lotes

In [4]:
class BatchGenerator():
    def __init__(self, instances, anchors, labels, batch_size=1, shuffle=True):
        self.instances          = instances
        self.batch_size         = batch_size
        self.labels             = labels
        self.anchors            = [BoundBox(0, 0, anchors[2*i], anchors[2*i+1]) for i in range(len(anchors)//2)]

        if shuffle:
            np.random.shuffle(self.instances)      
            
    def num_classes(self):
        return len(self.labels)

    def size(self):
        return len(self.instances)    

    def get_anchors(self):
        anchors = []

        for anchor in self.anchors:
            anchors += [anchor.xmax, anchor.ymax]

        return anchors

    def load_annotation(self, i):
        annots = []

        for obj in self.instances[i]['object']:
            annot = [obj['xmin'], obj['ymin'], obj['xmax'], obj['ymax'], self.labels.index(obj['name'])]
            annots += [annot]

        if len(annots) == 0: annots = [[]]

        return np.array(annots)

    def load_image(self, i):
        return cv2.imread(self.instances[i]['filename'])    

## Detection

In [5]:
def do_nms(boxes, nms_thresh):
    if len(boxes) > 0:
        nb_class = len(boxes[0].classes)
    else:
        return
        
    for c in range(nb_class):
        sorted_indices = np.argsort([-box.classes[c] for box in boxes])

        for i in range(len(sorted_indices)):
            index_i = sorted_indices[i]

            if boxes[index_i].classes[c] == 0: continue

            for j in range(i+1, len(sorted_indices)):
                index_j = sorted_indices[j]

                if bbox_iou(boxes[index_i], boxes[index_j]) >= nms_thresh:
                    boxes[index_j].classes[c] = 0
                    
def get_yolo_boxes(model, images, net_h, net_w, nms_thresh):
    batch_output, data = model.detect_image(Image.fromarray(images[0].astype('uint8')))
    boxes = []
    
    for bo in batch_output:
        b = [0]*2
        b[bo[4]] = bo[5]
        box = bo[:4] + [bo[5]] + [b]
        boxes.append(BoundBox(box[0], box[1], box[2], box[3], box[4], box[5]))

    # image_h, image_w, _ = images[0].shape
    # correct_yolo_boxes(boxes, image_h, image_w, net_h, net_w)

    do_nms(boxes, nms_thresh)
    return [boxes]


In [6]:
def detection(model, generator, nms_thresh=0.5, net_h=416, net_w=416):  
    # gather all detections and annotations
    all_detections     = [[None for i in range(generator.num_classes())] for j in range(generator.size())]
    all_annotations    = [[None for i in range(generator.num_classes())] for j in range(generator.size())]

    for i in range(generator.size()):
        raw_image = [generator.load_image(i)]

        # make the boxes and the labels
        pred_boxes = get_yolo_boxes(model, raw_image, net_h, net_w, nms_thresh)[0]

        score = np.array([box.get_score() for box in pred_boxes])
        pred_labels = np.array([box.label for box in pred_boxes])        
        
        if len(pred_boxes) > 0:
            pred_boxes = np.array([[box.xmin, box.ymin, box.xmax, box.ymax, box.get_score()] for box in pred_boxes]) 
        else:
            pred_boxes = np.array([[]])  
        
        # sort the boxes and the labels according to scores
        score_sort = np.argsort(-score)
        pred_labels = pred_labels[score_sort]
        pred_boxes  = pred_boxes[score_sort]
        
        # copy detections to all_detections
        for label in range(generator.num_classes()):
            all_detections[i][label] = pred_boxes[pred_labels == label, :]

        annotations = generator.load_annotation(i)
        
        # copy detections to all_annotations
        for label in range(generator.num_classes()):
            all_annotations[i][label] = annotations[annotations[:, 4] == label, :4].copy()
            
    return all_detections, all_annotations

## Evaluation

In [7]:
def compute_overlap(a, b):
    """
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    Parameters
    ----------
    a: (N, 4) ndarray of float
    b: (K, 4) ndarray of float
    Returns
    -------
    overlaps: (N, K) ndarray of overlap between boxes and query_boxes
    """
    area = (b[:, 2] - b[:, 0]) * (b[:, 3] - b[:, 1])

    iw = np.minimum(np.expand_dims(a[:, 2], axis=1), b[:, 2]) - np.maximum(np.expand_dims(a[:, 0], 1), b[:, 0])
    ih = np.minimum(np.expand_dims(a[:, 3], axis=1), b[:, 3]) - np.maximum(np.expand_dims(a[:, 1], 1), b[:, 1])

    iw = np.maximum(iw, 0)
    ih = np.maximum(ih, 0)

    ua = np.expand_dims((a[:, 2] - a[:, 0]) * (a[:, 3] - a[:, 1]), axis=1) + area - iw * ih

    ua = np.maximum(ua, np.finfo(float).eps)

    intersection = iw * ih

    return intersection / ua  

def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.

    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.], recall, [1.]))
    mpre = np.concatenate(([0.], precision, [0.]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap  

In [8]:
def evaluation(all_detections, all_annotations, generator, iou_threshold=0.5):
    average_precisions = []
    
    for label in range(generator.num_classes()):
        false_positives = np.zeros((0,))
        true_positives  = np.zeros((0,))
        scores          = np.zeros((0,))
        num_annotations = 0.0

        for i in range(generator.size()):
            detections           = all_detections[i][label]
            annotations          = all_annotations[i][label]
            num_annotations     += annotations.shape[0]
            detected_annotations = []

            for d in detections:
                scores = np.append(scores, d[4])

                if annotations.shape[0] == 0: # Si no hay anotación de esa detección es un falso positivo
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    continue

                overlaps            = compute_overlap(np.expand_dims(d, axis=0), annotations) # IOU, tiene el consideración todas las anotaciones
                assigned_annotation = np.argmax(overlaps, axis=1) # Se queda con la anotación que maximiza el IOU
                max_overlap         = overlaps[0, assigned_annotation] # Se queda con el valor del IOU se esta anotación

                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations: # Comprueba si esa anotación no ha sido ya asignada a una detección (además de comprobar que el IOU supera un cierto umbral). Las detecciones están ordenadas por score descendente por lo que se quedaría primero la que tiene mayor score (aunque luego pueda tener menor IoU).
                    false_positives = np.append(false_positives, 0)
                    true_positives  = np.append(true_positives, 1)
                    detected_annotations.append(assigned_annotation) # Guarda la anotación para que no pueda volver a ser usada
                else: # IOU por debajo del umbral o anotación ya utilizada
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0) 

        # no annotations -> AP for this class is 0 (is this correct?)
        if num_annotations == 0:
            average_precisions[label] = 0
            continue

        # sort by score (Esto lo hace para ser consistente con los vectores de anotación y detección)
        indices         = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives  = true_positives[indices]
        annotations_pending = num_annotations - np.sum(true_positives)

        # compute false positives and true positives (Esto es lo mismo que sumar unos y ceros de cada una de los vectores pero se hace así para computar el AP)
        false_positives = np.cumsum(false_positives)
        true_positives  = np.cumsum(true_positives)

        # compute recall and precision (Y el F1)
        recall    = true_positives / num_annotations # Es lo mismo que dividir entre TP + FN porque la suma de ambas tiene que ser el número de anotaciones (se detecten o no)
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)
        f1 = 2 * (precision * recall) / (precision + recall)

        # compute average precision
        average_precision  = compute_ap(recall, precision)
        average_precisions.append({'label': generator.labels[label], 'AP': average_precision, 'recall': recall[-1], 'precision': precision[-1], 'support': num_annotations, 'TP':true_positives[-1], 'FP': false_positives[-1], 'FN': annotations_pending})

    return average_precisions

In [9]:
def evaluation_v2(all_detections, all_annotations, generator, confidence_threshold = 0.5, iou_threshold=0.5):
    average_precisions = []
    
    for label in range(generator.num_classes()):
        false_positives = np.zeros((0,))
        true_positives  = np.zeros((0,))
        false_negatives = np.zeros((0,))
        true_negatives  = np.zeros((0,))
        scores          = np.zeros((0,))
        num_annotations = 0.0
        annotations_pending = 0.0

        for i in range(generator.size()):
            detections           = all_detections[i][label]
            annotations          = all_annotations[i][label]
            num_annotations     += annotations.shape[0]
            detected_annotations = []

            for d in detections:
                scores = np.append(scores, d[4])

                if annotations.shape[0] == 0: # Si no hay anotación
                    # index = int(not bool(label))
                    # all_annotations[i][index].shape[0] == 0
                    if d[4] < confidence_threshold: # El score no supera un umbral, verdadero negativo
                        false_positives = np.append(false_positives, 0)
                        true_positives  = np.append(true_positives, 0)
                        false_negatives = np.append(false_negatives, 0)
                        true_negatives  = np.append(true_negatives, 1) 
                    else: # El score es alto, falso positivo
                        false_positives = np.append(false_positives, 1)
                        true_positives  = np.append(true_positives, 0)
                        false_negatives = np.append(false_negatives, 0)
                        true_negatives  = np.append(true_negatives, 0)
                    continue
                    
                overlaps            = compute_overlap(np.expand_dims(d, axis=0), annotations) # IOU, tiene el consideración todas las anotaciones
                assigned_annotation = np.argmax(overlaps, axis=1) # Se queda con la anotación que maximiza el IOU
                max_overlap         = overlaps[0, assigned_annotation] # Se queda con el valor del IOU se esta anotación
                
                if assigned_annotation in detected_annotations:
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    false_negatives = np.append(false_negatives, 0)
                    true_negatives  = np.append(true_negatives, 0)
                    continue
                
                if max_overlap >= iou_threshold and d[4] >= confidence_threshold: # Comprueba que el IOU supera un cierto umbral de igual modo que la precisión en la clasificación debe superar otro
                    false_positives = np.append(false_positives, 0)
                    true_positives  = np.append(true_positives, 1)
                    false_negatives = np.append(false_negatives, 0)
                    true_negatives  = np.append(true_negatives, 0)
                    detected_annotations.append(assigned_annotation)
                else: # IOU por debajo del umbral o precisión en la clasificación por debajo de su umbral
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    false_negatives = np.append(false_negatives, 0)
                    true_negatives  = np.append(true_negatives, 0)
                    
            annotations_pending = (annotations_pending + annotations.shape[0]) - len(detected_annotations) 
            
        # no annotations -> AP for this class is 0 (is this correct?)
        if num_annotations == 0:
            average_precisions[label] = 0
            continue

        # sort by score (Esto lo hace para ser consistente con los vectores de anotación y detección)
        indices         = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives  = true_positives[indices]
        true_negatives  = true_negatives[indices]

        # compute false positives and true positives (Esto es lo mismo que sumar unos y ceros de cada una de los vectores pero se hace así para computar el AP)
        false_positives = np.cumsum(false_positives)
        true_positives  = np.cumsum(true_positives)
        true_negatives  = np.cumsum(true_negatives)

        # compute recall and precision (Y el F1)
        recall    = true_positives / num_annotations # Es lo mismo que dividir entre TP + FN porque la suma de ambas tiene que ser el número de anotaciones (se detecten o no)
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)
        f1 = 2 * (precision * recall) / (precision + recall)

        # compute average precision
        average_precision  = compute_ap(recall, precision)
        average_precisions.append({'label': generator.labels[label], 'AP': average_precision, 'recall': recall[-1], 'precision': precision[-1], 'f1': f1[-1], 'support': num_annotations, 'TP':true_positives[-1], 'FP': false_positives[-1], 'TN': true_negatives[-1], 'FN': annotations_pending})

    return average_precisions

# Evaluación

## Carga de modelo y de datos de test

In [10]:
os.chdir('/home/ubuntu/tfm')
config_path = './utils/config.json'

with open(config_path) as config_buffer:    
    config = json.loads(config_buffer.read())

instances = pckl.load(open(config['model']['dataset_folder'], 'rb'))
labels = config['model']['labels']
labels = sorted(labels)

valid_generator = BatchGenerator(
    instances           = instances,
    anchors             = config['model']['anchors'],   
    labels              = sorted(config['model']['labels']),
)

infer_model = load_model(config['train']['model_folder'], config['train']['classes_path'], config['train']['anchors_path'])

/home/ubuntu/tfm/standalone/models/82f298920f8646ef89bd4313ef8202d9/artifacts/model/data/model.h5 model, anchors, and classes loaded in 15.63sec.


## Test

In [11]:
all_detections, all_annotations = detection(infer_model, valid_generator)

(416, 416, 3)
Found 2 boxes for img
Panel 0.95 (460, 231) (1661, 646)
Dedo 0.89 (293, 428) (1037, 1080)
Time spent: 2.900sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.79 (978, 224) (1398, 371)
Panel 0.86 (0, 124) (321, 337)
Panel 0.96 (1468, 242) (1597, 354)
Dedo 0.85 (888, 335) (1212, 698)
Time spent: 0.434sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.74 (824, 104) (1292, 406)
Dedo 0.96 (833, 306) (988, 533)
Time spent: 0.423sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.68 (866, 405) (1074, 483)
Dedo 0.69 (880, 467) (1211, 860)
Time spent: 0.436sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.81 (1428, 371) (1704, 427)
Panel 0.90 (1000, 370) (1230, 444)
Panel 0.97 (308, 224) (690, 447)
Dedo 0.97 (901, 393) (1074, 625)
Time spent: 0.419sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.60 (711, 137) (1110, 323)
Panel 0.63 (1154, 213) (1524, 362)
Panel 0.67 (20, 52) (624, 354)
Dedo 0.70 (1279, 317) (1496, 656)
Time spent: 0.424sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.83 (0, 453

Found 4 boxes for img
Panel 0.66 (966, 272) (1472, 427)
Panel 0.82 (227, 249) (451, 345)
Panel 0.93 (512, 254) (653, 361)
Dedo 0.72 (1123, 565) (1467, 914)
Time spent: 0.421sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.61 (1537, 457) (1766, 604)
Panel 0.98 (485, 204) (1525, 479)
Dedo 0.93 (751, 295) (851, 650)
Time spent: 0.418sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.70 (1112, 349) (1492, 470)
Panel 0.89 (494, 260) (811, 451)
Panel 0.97 (812, 257) (1140, 465)
Dedo 0.97 (759, 400) (911, 696)
Time spent: 0.417sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.93 (0, 235) (1920, 625)
Dedo 0.95 (1342, 532) (1499, 773)
Time spent: 0.423sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.98 (31, 475) (207, 551)
Panel 0.99 (275, 439) (428, 510)
Dedo 0.96 (1041, 556) (1216, 753)
Time spent: 0.419sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.76 (931, 393) (1310, 476)
Panel 0.91 (1583, 446) (1785, 518)
Dedo 0.91 (493, 462) (790, 876)
Time spent: 0.419sec
(416, 416, 3)
Found 2 boxes for img
P

Found 5 boxes for img
Panel 0.61 (985, 239) (1412, 434)
Panel 0.75 (52, 605) (182, 712)
Panel 0.89 (564, 357) (993, 534)
Panel 0.92 (216, 497) (395, 622)
Dedo 0.61 (1085, 332) (1243, 615)
Time spent: 0.417sec
(416, 416, 3)
Found 5 boxes for img
Panel 0.72 (1694, 211) (1834, 330)
Panel 0.72 (927, 195) (1113, 327)
Panel 0.72 (1163, 206) (1612, 345)
Panel 0.84 (0, 132) (554, 363)
Dedo 0.70 (978, 330) (1210, 779)
Time spent: 0.415sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.62 (1070, 451) (1311, 539)
Dedo 0.71 (595, 455) (790, 715)
Time spent: 0.408sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.53 (35, 288) (499, 422)
Panel 0.79 (1056, 312) (1520, 457)
Dedo 0.76 (290, 336) (573, 672)
Dedo 0.79 (372, 397) (525, 630)
Time spent: 0.415sec
(416, 416, 3)
Found 5 boxes for img
Panel 0.77 (1686, 230) (1831, 354)
Panel 0.82 (1179, 206) (1591, 362)
Panel 0.83 (927, 197) (1102, 335)
Panel 0.87 (0, 124) (552, 351)
Dedo 0.86 (834, 326) (1121, 730)
Time spent: 0.421sec
(416, 416, 3)
Found 2 boxes f

Found 3 boxes for img
Panel 0.60 (81, 363) (304, 455)
Panel 0.87 (505, 325) (1053, 475)
Dedo 0.78 (1184, 478) (1494, 996)
Time spent: 0.414sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.98 (338, 390) (1395, 605)
Dedo 0.88 (919, 438) (1638, 1080)
Time spent: 0.412sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.94 (0, 896) (600, 1080)
Panel 0.95 (1133, 709) (1685, 945)
Panel 0.95 (607, 775) (1149, 1070)
Dedo 0.54 (1319, 294) (1851, 698)
Time spent: 0.421sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.95 (105, 113) (807, 486)
Panel 0.96 (1202, 484) (1537, 650)
Dedo 0.55 (925, 516) (1251, 1042)
Time spent: 0.418sec
(416, 416, 3)
Found 5 boxes for img
Panel 0.72 (722, 347) (1104, 467)
Panel 0.85 (1751, 414) (1909, 532)
Panel 0.96 (98, 263) (437, 446)
Panel 0.97 (441, 257) (764, 462)
Dedo 0.93 (1242, 548) (1438, 729)
Time spent: 0.414sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.91 (65, 479) (276, 535)
Panel 0.93 (335, 424) (500, 497)
Dedo 0.95 (1109, 510) (1228, 737)
Time spent: 0.421sec

Found 6 boxes for img
Panel 0.55 (932, 363) (1186, 540)
Panel 0.74 (1203, 513) (1428, 604)
Panel 0.87 (711, 264) (928, 445)
Panel 0.92 (1616, 597) (1725, 688)
Panel 0.98 (401, 196) (696, 380)
Dedo 0.96 (911, 441) (1086, 712)
Time spent: 0.427sec
(416, 416, 3)
Found 5 boxes for img
Panel 0.65 (800, 236) (1227, 356)
Panel 0.75 (1302, 254) (1727, 354)
Panel 0.83 (18, 282) (219, 393)
Panel 0.88 (305, 255) (545, 320)
Dedo 0.92 (985, 421) (1174, 702)
Time spent: 0.424sec
(416, 416, 3)
Found 5 boxes for img
Panel 0.69 (507, 249) (927, 368)
Panel 0.79 (1416, 448) (1654, 541)
Panel 0.88 (1171, 349) (1350, 453)
Panel 0.92 (1727, 473) (1909, 560)
Dedo 0.84 (900, 393) (1086, 574)
Time spent: 0.418sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.63 (774, 100) (1335, 369)
Panel 0.88 (1353, 29) (1920, 266)
Panel 0.89 (430, 210) (802, 384)
Dedo 0.78 (327, 317) (632, 735)
Time spent: 0.416sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.89 (8, 426) (236, 497)
Panel 0.97 (273, 441) (474, 527)
Panel 0.98 (

...

Found 3 boxes for img
Panel 0.85 (291, 206) (1424, 452)
Panel 0.87 (1526, 337) (1918, 472)
Dedo 0.92 (1065, 404) (1254, 742)
Time spent: 0.415sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.99 (306, 454) (1409, 670)
Time spent: 0.417sec
(416, 416, 3)
Found 1 boxes for img
Dedo 0.92 (885, 539) (1078, 874)
Time spent: 0.411sec
(416, 416, 3)
Found 7 boxes for img
Panel 0.61 (957, 300) (1220, 381)
Panel 0.63 (228, 302) (547, 439)
Panel 0.65 (574, 322) (926, 475)
Panel 0.73 (1505, 250) (1645, 305)
Panel 0.74 (1650, 229) (1711, 301)
Panel 0.98 (1299, 256) (1461, 339)
Dedo 0.84 (1048, 331) (1228, 616)
Time spent: 0.416sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.63 (163, 63) (757, 348)
Panel 0.78 (1128, 324) (1420, 509)
Panel 0.84 (732, 218) (1146, 432)
Dedo 0.90 (1228, 419) (1379, 713)
Time spent: 0.481sec
(416, 416, 3)
Found 1 boxes for img
Dedo 0.79 (951, 544) (1285, 866)
Time spent: 0.439sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.51 (1436, 334) (1920, 478)
Panel 0.98 (211, 180) (1

Found 1 boxes for img
Panel 0.99 (599, 326) (1548, 721)
Time spent: 0.417sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.81 (1138, 34) (1712, 275)
Panel 0.92 (778, 196) (1124, 476)
Panel 0.94 (518, 340) (774, 566)
Dedo 0.66 (643, 450) (913, 824)
Time spent: 0.418sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.58 (180, 379) (439, 463)
Panel 0.69 (616, 380) (1028, 467)
Dedo 0.92 (1088, 520) (1382, 923)
Time spent: 0.418sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.55 (1445, 377) (1895, 516)
Panel 0.98 (92, 191) (1565, 470)
Dedo 0.87 (1005, 468) (1163, 807)
Time spent: 0.422sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.84 (39, 0) (1310, 237)
Time spent: 0.422sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.74 (225, 611) (755, 797)
Panel 0.87 (750, 591) (1098, 733)
Dedo 0.51 (640, 213) (893, 610)
Time spent: 0.413sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.80 (1534, 192) (1894, 337)
Panel 0.97 (148, 180) (918, 384)
Dedo 0.64 (339, 314) (597, 732)
Time spent: 0.508sec
(416, 416, 3)
F

Found 9 boxes for img
Panel 0.64 (97, 372) (157, 431)
Panel 0.73 (1544, 289) (1896, 400)
Panel 0.76 (195, 378) (279, 430)
Panel 0.89 (1130, 320) (1314, 400)
Panel 0.90 (807, 378) (1056, 456)
Panel 0.93 (321, 373) (471, 422)
Panel 0.94 (560, 377) (737, 430)
Dedo 0.50 (700, 453) (842, 590)
Dedo 0.85 (706, 374) (877, 713)
Time spent: 0.430sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.57 (1507, 456) (1920, 654)
Panel 0.97 (412, 564) (1515, 991)
Dedo 0.71 (1024, 296) (1273, 694)
Time spent: 0.413sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.71 (1612, 212) (1915, 392)
Panel 0.95 (214, 162) (1037, 381)
Dedo 0.65 (338, 502) (616, 912)
Time spent: 0.413sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.59 (146, 256) (618, 402)
Panel 0.88 (653, 192) (1051, 330)
Time spent: 0.439sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.82 (1129, 311) (1472, 436)
Panel 0.87 (787, 217) (1150, 440)
Panel 0.87 (507, 220) (795, 396)
Dedo 0.97 (687, 375) (874, 631)
Time spent: 0.635sec
(416, 416, 3)
Found 3 box

Found 5 boxes for img
Panel 0.67 (987, 23) (1463, 373)
Panel 0.70 (645, 230) (1058, 424)
Panel 0.79 (1336, 0) (1892, 232)
Panel 0.94 (320, 355) (666, 522)
Dedo 0.68 (986, 295) (1213, 662)
Time spent: 0.420sec
(416, 416, 3)
Found 1 boxes for img
Panel 0.96 (527, 406) (1280, 725)
Time spent: 0.414sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.93 (903, 3) (1584, 227)
Dedo 0.95 (1095, 225) (1283, 482)
Time spent: 0.414sec
(416, 416, 3)
Found 5 boxes for img
Panel 0.78 (1441, 247) (1571, 354)
Panel 0.80 (11, 179) (303, 336)
Panel 0.81 (960, 221) (1366, 368)
Dedo 0.50 (963, 338) (1275, 712)
Dedo 0.85 (1030, 368) (1214, 652)
Time spent: 0.421sec
(416, 416, 3)
Found 2 boxes for img
Panel 0.98 (361, 345) (1368, 609)
Dedo 0.66 (966, 462) (1623, 1080)
Time spent: 0.436sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.77 (0, 112) (854, 464)
Panel 0.94 (1356, 77) (1920, 408)
Panel 0.98 (605, 0) (1284, 286)
Time spent: 0.425sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.64 (756, 123) (1240, 361)
Pan

Found 6 boxes for img
Panel 0.59 (753, 329) (1145, 477)
Panel 0.59 (1772, 401) (1920, 517)
Panel 0.95 (140, 257) (473, 431)
Panel 0.98 (458, 234) (805, 460)
Dedo 0.67 (1320, 506) (1513, 734)
Dedo 0.67 (1337, 504) (1443, 686)
Time spent: 0.418sec
(416, 416, 3)
Found 6 boxes for img
Panel 0.55 (808, 225) (1128, 379)
Panel 0.81 (1720, 396) (1905, 483)
Panel 0.91 (79, 83) (457, 281)
Panel 0.96 (467, 101) (782, 338)
Dedo 0.62 (934, 346) (1257, 655)
Dedo 0.90 (1031, 389) (1207, 613)
Time spent: 0.426sec
(416, 416, 3)
Found 4 boxes for img
Panel 0.86 (227, 169) (1714, 428)
Panel 0.88 (1531, 334) (1916, 474)
Dedo 0.68 (743, 367) (1066, 669)
Dedo 0.88 (820, 391) (974, 618)
Time spent: 0.413sec
(416, 416, 3)
Found 3 boxes for img
Panel 0.82 (0, 137) (93, 402)
Panel 0.93 (194, 154) (1920, 527)
Dedo 0.90 (1176, 223) (1905, 1049)
Time spent: 0.413sec
(416, 416, 3)
Found 5 boxes for img
Panel 0.75 (1149, 337) (1405, 395)
Panel 0.88 (1660, 344) (1920, 412)
Panel 0.92 (67, 156) (482, 352)
Panel 0.97 (

In [12]:
average_precisions = evaluation(all_detections, all_annotations, valid_generator)

## Procesar salida

In [13]:
items = 0
precision = 0
for average_precision in average_precisions:    
    items += 1 
    precision += average_precision['AP']
display(pd.DataFrame(average_precisions))
print('mAP: {:.4f}'.format(precision / items))

,label,AP,recall,precision,support,TP,FP,FN
0,Dedo,0.732858,0.815476,0.822000,504.0,411.0,89.0,93.0
1,Panel,0.710166,0.731740,0.933108,1506.0,1102.0,79.0,404.0


mAP: 0.7215


## Experimental

In [14]:
average_precisions = evaluation_v2(all_detections, all_annotations, valid_generator)

In [15]:
items = 0
precision = 0
for average_precision in average_precisions:    
    items += 1 
    precision += average_precision['AP']
display(pd.DataFrame(average_precisions))
print('mAP: {:.4f}'.format(precision / items))

,label,AP,recall,precision,f1,support,TP,FP,TN,FN
0,Dedo,0.732858,0.815476,0.822000,0.818725,504.0,411.0,89.0,0.0,93.0
1,Panel,0.710166,0.731740,0.933108,0.820246,1506.0,1102.0,79.0,0.0,404.0


mAP: 0.7215


## Prueba completa

In [18]:
import mlflow
import os
import shutil
import boto3
from datetime import datetime


S3_CLIENT = boto3.resource('s3')

mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))
MLFLOW_CLIENT = mlflow.tracking.MlflowClient()

REGISTERED_MODELS = ["Hands"]
MODELS = {}


def downlod_model(bucket_name, remoteDirectory_name):

    bucket = S3_CLIENT.Bucket(bucket_name)

    for obj in bucket.objects.filter(Prefix=remoteDirectory_name):
        if not os.path.exists(os.path.dirname(obj.key)):
            os.makedirs(os.path.dirname(obj.key))
        bucket.download_file(obj.key, obj.key)


def update_models(version=-1, remove_old_versions=False):

    update = {}

    for model_name in REGISTERED_MODELS:
        model = None
        update[model_name] = 0
        for mv in MLFLOW_CLIENT.search_model_versions(f"name='{model_name}'"):
            mv_bckp = mv
            mv = dict(mv)
            if version == mv['version'] or (version == -1 and mv['current_stage'] == 'Production'):
                mv['last_updated_timestamp'] = str(datetime.fromtimestamp(int(mv['last_updated_timestamp'] / 1000)))
                bucket = mv['source'].split('//')[1].split('/')[0]
                folder = mv['source'].split('//')[1].split('/')[1]
                if os.path.exists(os.path.join('./models', folder)):
                    print("Load existing model...")
                    model = os.path.join(os.path.join('./models', folder), "artifacts/model/data/model.h5")
                else:
                    print("Downloading model...")
                    downlod_model(bucket, folder)
                    model = os.path.join(os.path.join('./models', folder), "artifacts/model/data/model.h5")
                    if remove_old_versions and os.path.exists('./models'):
                        shutil.rmtree('./models')
                    if not os.path.exists('./models'):
                        os.mkdir('./models')
                    shutil.move(os.path.join(os.getcwd(), folder), './models')
                    update[model_name] = 1
                print("Using model {name} v{version} ({current_stage}) updated at {last_updated_timestamp}".format(**mv))
                #response = {k: v for k, v in mv.items() if v}
                break
        if model:
            MODELS[model_name] = (model, mv_bckp)

    return update


def get_model(model_name):

    return MODELS.get(model_name, None)

/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
%%capture
os.chdir('/home/ubuntu/tfm/standalone')
config_path = '../utils/config.json'

with open(config_path) as config_buffer:    
    config = json.loads(config_buffer.read())

instances = pckl.load(open(config['model']['dataset_folder'], 'rb'))
labels = config['model']['labels']
labels = sorted(labels)

valid_generator = BatchGenerator(
    instances           = instances,
    anchors             = config['model']['anchors'],   
    labels              = sorted(config['model']['labels']),
)

results = []
results_ = []
versions = [10, 11]
for version in versions:
    update_models(version)
    model_path, model_meta = get_model('Hands')
    print(MLFLOW_CLIENT.get_run(model_meta.run_id))
    infer_model = load_model(model_path, config['train']['classes_path'], config['train']['anchors_path'])
    all_detections, all_annotations = detection(infer_model, valid_generator)
    average_precisions = evaluation(all_detections, all_annotations, valid_generator)
    items = 0
    precision = 0
    for average_precision in average_precisions:    
        items += 1 
        precision += average_precision['AP']
    results.append(pd.DataFrame(average_precisions))
    results_.append('mAP: {:.4f}'.format(precision / items))
    display(pd.DataFrame(average_precisions))
    print('mAP: {:.4f}'.format(precision / items))

In [27]:
display(results[1])

,label,AP,recall,precision,support,TP,FP,FN
0,Dedo,0.732858,0.815476,0.822000,504.0,411.0,89.0,93.0
1,Panel,0.710165,0.731740,0.933108,1506.0,1102.0,79.0,404.0


In [28]:
1102+404

/home/ubuntu/miniconda3/envs/tfm/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


1506